In [99]:
!pip install pandas

In [100]:
import pandas as pd

### Importação dos arquivos CSV

Abaixo importei todos os arquivos .CSV do dataset "Brazilian E-commerce Public Data ny Olist" utilizando a biblioteca pandas. Cada DataFrame representa uma das entidades principais e seus relacionamentos.

In [102]:
# Pedidos e Detalhes 
df_orders = pd.read_csv("olist_orders_dataset.csv", sep=",")
df_order_items = pd.read_csv("olist_order_items_dataset.csv", sep=",")
df_payments = pd.read_csv("olist_order_payments_dataset.csv", sep=",")
df_reviews = pd.read_csv("olist_order_reviews_dataset.csv", sep=",")

#Entidades Relacionadas
df_customers = pd.read_csv("olist_customers_dataset.csv", sep=",")
df_products = pd.read_csv("olist_products_dataset.csv", sep=",")
df_sellers = pd.read_csv("olist_sellers_dataset.csv", sep=",")
df_category = pd.read_csv("product_category_name_translation.csv", sep=",")
df_geolocation = pd.read_csv("olist_geolocation_dataset.csv", sep=",")

### Limpeza dos Dados

Nesta etapa do código, realizei a verificação dos valores nulos em todos os dataframes.
1. Convertir as colunas com valores que continhas datas para o tipo datetime para facilitar análises temporais;
2. Removi valores duplicados, onde foi necessario;
3. No dataframe (df_orders) mantive os valores nulos nas colunas (order_delivered_carrier_date) e (order_delivered_customer_date), pois percebi que a ausencia desses dados, poderia significar que o pedido deve ter sido cancelado;
4. No dataframe (df_products), removi os dados da coluna (product_category_name), pois não são úteis para analises futuras.
5. Realizei o tratamento de strings nos campos textuais, removendo espaços em brancos indevidos antes e depos dos valores. Isso ajuda a evitar erros no momento de realizar joins, agrupamentos ou filtros;
6. Converti dados de das colunas categoricas que estavam no tipo objetos para tipos categoricos para tornar as analises mais corretas;
7. Verifiquei valores invalidos ou inesperados, como: status de pedidos inexisitentes, evitando distorções em analises futuras;
8. Verifiquei se havia inconsistencias de data, na data de entrega antes da compra, assim, evitando conclusões erradas sobre prazos logisticos e melhorando a confiabilidade das metricas temporais (estimativa de entrega dos produtos);
9. Analisei as coordenadas geográficas nulas ou inválidas. Assim, mantendo seguro a precisão em analises geoespaciais e mapas, evitando pontos imprecisos ou não existentes;
10. Verifiquei se a tradução de categorias de produtos PT-BR para EN-US estava funcionando corretamente, para assegurar que o publico que não fala português, entendam os dados contidos na coluna de categorias.
11. Encontrei algumas

Verificando a quantidade de valores nulos e duplicados em todos os dataframes

In [106]:
# Verificando valores nulos e duplicatas em todos os DataFrames
dataframes = {
    'orders': df_orders,
    'order_items': df_order_items,
    'payments': df_payments,
    'reviews': df_reviews,
    'customers': df_customers,
    'sellers': df_sellers,
    'products': df_products,
    'category': df_category,
    'geolocation': df_geolocation
}

for name, df in dataframes.items():
    print(f"\n {name.upper()}")
    print("Quantidade de linhas com valores nulos:\n", df.isnull().sum())
    print("Linhas com valores duplicados:", df.duplicated().sum())


 ORDERS
Quantidade de linhas com valores nulos:
 order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64
Linhas com valores duplicados: 0

 ORDER_ITEMS
Quantidade de linhas com valores nulos:
 order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64
Linhas com valores duplicados: 0

 PAYMENTS
Quantidade de linhas com valores nulos:
 order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64
Linhas com valores duplicados: 0

 REVIEWS
Quantidade de linhas com valores nulos:
 review_id                      0
order_id    

Convertendo as colunas de data para datetime, serve para podermos utilizar análises temporais mais eficazes.

In [108]:
colunas_data = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 
             'order_delivered_customer_date', 'order_estimated_delivery_date']
df_orders[colunas_data] = df_orders[colunas_data].apply(pd.to_datetime)

# Nenhuma coluna precisa ser preenchida com média ou mediana, pois datas ausentes em (approved ou delivered) podem representar cancelamentos dos pedidos.

# ### removendo valores duplicados dos dataframes ###

# removendo linhas duplicadas de orders
df_orders.drop_duplicates(inplace=True)
# removendo linhas duplicadas de order_items
df_order_items.drop_duplicates(inplace=True)
# removendo linhas duplicadas de payments
df_payments.drop_duplicates(inplace=True)

# Converter colunas de data do dataframa reviews
df_reviews['review_creation_date'] = pd.to_datetime(df_reviews['review_creation_date'])
df_reviews['review_answer_timestamp'] = pd.to_datetime(df_reviews['review_answer_timestamp'])
# removendo linhas duplicadas de reviews
df_reviews.drop_duplicates(inplace=True)

# removendo linhas duplicadas de customers
df_customers.drop_duplicates(inplace=True)
# removendo linhas duplicadas de sellers
df_sellers.drop_duplicates(inplace=True)

# Já que sabemos que existem linhas nulas no dataframe df_products, então teremos que usar uma estageria de manter ou remover os nulos
# isso vai depender de como nos iremos utilizar os dados futuramente.

# Verificar colunas com nulos do dataframe df_products. Retorna um valor boleano True or False. E soma a quantidade de dados nulos.
print(df_products.isnull().sum())

# Removemos do dataframe df_products todas as linhas que têm valor nulo apenas na coluna product_category_name
# Esse processo é importante para análises por categoria. Manter valores nulos nela pode atrapalhar agregações e visualizações.
df_products.dropna(subset=['product_category_name'], inplace=True)

# Removemos todas as linhas duplicadas da tabela df_products, pois o pandas considera como duplicadas as linhas que possuem valores iguais em todas as colunas.
df_products.drop_duplicates(inplace=True)
# removendo linhas duplicadas de category
df_category.drop_duplicates(inplace=True)
# Geolocation pode ter duplicatas porque existem várias entradas por cidade
df_geolocation.drop_duplicates(inplace=True)

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64


Tratamento de Strings

In [110]:
# Essa função, faz o tratamento de remoção de espaços em brancos nas colunas do tipo string.
def remover_espacos(df):
    colunas_str = df.select_dtypes(include = 'object').columns
    df[colunas_str] = df[colunas_str].apply(lambda x: x.str.strip())
    return df

# Aplicar nos dataframes
df_orders = remover_espacos(df_orders)
df_order_items = remover_espacos(df_order_items)
df_payments = remover_espacos(df_payments)
df_reviews = remover_espacos(df_reviews)
df_customers = remover_espacos(df_customers)
df_sellers = remover_espacos(df_sellers)
df_products = remover_espacos(df_products)
df_category = remover_espacos(df_category)
df_geolocation = remover_espacos(df_geolocation)


In [175]:
# Exemplo para evitar conflitos entre nomes de cidades.
df2 = df_sellers[["seller_city", "seller_state"]]
df2

,seller_city,seller_state
0,campinas,SP
1,mogi guacu,SP
2,rio de janeiro,RJ
3,sao paulo,SP
4,braganca paulista,SP
...,...,...
3090,sarandi,PR
3091,palhoca,SC
3092,sao paulo,SP
3093,pelotas,RS


Verificando os tipos de inconsistencias. Para isso, vamos verificar se os tipos das colunas IDs e colunas categóricas, estão corretas. Então convertemos as colunas de categorias para o tipo category e verificamos os valores inválidos ou não esperados nessas colunas.

In [144]:
# Convertendo 
category_columns = ['order_status', 'payment_type', 'review_score',
               'product_category_name', 'customer_state', 'seller_state']

for col in category_columns:
    for df in [df_orders, df_payments, df_reviews, df_products, df_customers, df_sellers]:
        if col in df.columns:
            df[col] = df[col].astype('category')
            
# Verificando a quantidade de valores invalidos ou inesperados das colunas categoricas
print("###### orders ######\n")
print(df_orders['order_status'].value_counts())
print("\n###### payments ######\n")
print(df_payments['payment_type'].value_counts())
print("\n###### reviews ######\n")
print(df_reviews['review_score'].value_counts())

###### orders ######

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

###### payments ######

payment_type
credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
not_defined        3
Name: count, dtype: int64

###### reviews ######

review_score
5    57328
4    19142
1    11424
3     8179
2     3151
Name: count, dtype: int64


Verificando colunas que contem valores zero, mas não deveriam.
Penso em filtrar se parecerem inválidos, ou manter caso forem pedidos gratuitos ou testes.

In [115]:
print(df_payments[df_payments['payment_value'] == 0])

                                order_id  payment_sequential payment_type  \
19922   8bcbe01d44d147f901cd3192671144db                   4      voucher   
36822   fa65dad1b0e818e3ccc5cb0e39231352                  14      voucher   
43744   6ccb433e00daae1283ccc956189c82ae                   4      voucher   
51280   4637ca194b6387e2d538dc89b124b0ee                   1  not_defined   
57411   00b1cb0320190ca0daa2c88b35206009                   1  not_defined   
62674   45ed6e85398a87c253db47c2d9f48216                   3      voucher   
77885   fa65dad1b0e818e3ccc5cb0e39231352                  13      voucher   
94427   c8c528189310eaa44a745b8d9d26908b                   1  not_defined   
100766  b23878b3e8eb4d25a158f57d96331b18                   4      voucher   

        payment_installments  payment_value  
19922                      1            0.0  
36822                      1            0.0  
43744                      1            0.0  
51280                      1            0.0  

Verificando se existem datas inconsistens de entrega antes da data de compra ou aprovação das ordens produtos.

In [117]:
df_orders[df_orders['order_delivered_customer_date'] < df_orders['order_purchase_timestamp']]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


Verificando se existem localizações inválidas (geolocalização). Verificando se há latitudes/longitudes muito discrepantes ou iguais a zero.

In [119]:
df_geolocation.query("geolocation_lat < -90 or geolocation_lat > 90 or geolocation_lng < -180 or geolocation_lng > 180")
df_geolocation.query("geolocation_lat == 0 or geolocation_lng == 0")

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state


Verificando se a cobertura da tradução dos produtos sem categorias. Já que removemos os valores nulos da coluna (product_category_name) do dataframe (df_products) anteriormente. Agora vamos verificar se a tradução do dataframe (product_category_name_translation) esta funcionando corretamente para o PT-BR, no dataframe (df_products).

In [217]:
df_bol = df_products[~df_products['product_category_name'].isin(df_category['product_category_name'])]
df_bol_2 = df_bol[["product_id", "product_category_name"]]
df_bol_2

,product_id,product_category_name
1581,0105b5323d24fc655f73052694dbbb3a,pc_gamer
5703,6fd83eb3e0799b775e4f946bd66657c0,portateis_cozinha_e_preparadores_de_alimentos
7176,5d923ead886c44b86845f69e50520c3e,portateis_cozinha_e_preparadores_de_alimentos
7326,6727051471a0fc4a0e7737b57bff2549,pc_gamer
8648,bed164d9d628cf0593003389c535c6e0,portateis_cozinha_e_preparadores_de_alimentos
10823,1220978a08a6b29a202bc015b18250e9,portateis_cozinha_e_preparadores_de_alimentos
13996,ae62bb0f95af63d64eae5f93dddea8d3,portateis_cozinha_e_preparadores_de_alimentos
15875,1954739d84629e7323a4295812a3e0ec,portateis_cozinha_e_preparadores_de_alimentos
16609,dbe520fb381ad695a7e1f2807d20c765,pc_gamer
17460,c7a3f1a7f9eef146cc499368b578b884,portateis_cozinha_e_preparadores_de_alimentos


Foram encontrados 2 classificações de categorias em (df_products) que não existem no dataframe de tradução (df_category).
Para corrigir isso, vamos inicialmente fazer um merge entre os dois data frames (df_products) e (df_category).

In [229]:
df_products = df_products.merge(
    df_category,
    on='product_category_name',
    how='left'
)

Logo após, vamos preencher os valores nulos na coluna com unknown.

In [241]:
df_products['product_category_name_english'] = df_products['product_category_name_english'].fillna('unknown')

Verificamos novamente se existem categorias não traduzidas.

In [262]:
df_products['product_category_name_english'].value_counts().loc['unknown']

13

Sim, atualmente existem 13 linhas sem tradução de categorias, vamos ver quais são.

In [269]:
df_products[df_products['product_category_name_english'] == 'unknown']

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
1581,0105b5323d24fc655f73052694dbbb3a,pc_gamer,59.0,621.0,4.0,2839.0,19.0,16.0,18.0,unknown
5703,6fd83eb3e0799b775e4f946bd66657c0,portateis_cozinha_e_preparadores_de_alimentos,52.0,280.0,1.0,1200.0,25.0,33.0,25.0,unknown
7176,5d923ead886c44b86845f69e50520c3e,portateis_cozinha_e_preparadores_de_alimentos,58.0,284.0,1.0,1200.0,25.0,33.0,25.0,unknown
7326,6727051471a0fc4a0e7737b57bff2549,pc_gamer,60.0,1532.0,3.0,650.0,16.0,22.0,20.0,unknown
8648,bed164d9d628cf0593003389c535c6e0,portateis_cozinha_e_preparadores_de_alimentos,54.0,382.0,2.0,850.0,30.0,21.0,22.0,unknown
10823,1220978a08a6b29a202bc015b18250e9,portateis_cozinha_e_preparadores_de_alimentos,46.0,280.0,1.0,1200.0,25.0,33.0,25.0,unknown
13996,ae62bb0f95af63d64eae5f93dddea8d3,portateis_cozinha_e_preparadores_de_alimentos,59.0,927.0,1.0,10600.0,40.0,20.0,38.0,unknown
15875,1954739d84629e7323a4295812a3e0ec,portateis_cozinha_e_preparadores_de_alimentos,58.0,792.0,4.0,750.0,30.0,30.0,30.0,unknown
16609,dbe520fb381ad695a7e1f2807d20c765,pc_gamer,60.0,840.0,6.0,800.0,18.0,22.0,22.0,unknown
17460,c7a3f1a7f9eef146cc499368b578b884,portateis_cozinha_e_preparadores_de_alimentos,52.0,1372.0,5.0,7350.0,40.0,30.0,23.0,unknown


Agora, por descargo de consciência, vamos verificar em percentual quantas categorias estão sendo traduzidas atualmente.

In [274]:
total = len(df_products)
traduzidas = df_products['product_category_name_english'].ne('unknown').sum()

cobertura = traduzidas / total * 100

print(f'Cobertura de tradução: {cobertura:.2f}%')

Cobertura de tradução: 99.96%
